## 파이썬을 활용하여 자동화
1. 메일 자동 발송
2. 엑셀 파일 제어

프로토콜 : 정보를 주고받을때의 전파 방식에 대한 규칙 <br>
이메일 교환 관련 프로토콜 : SMTP, POP3, IMAP

보안수준 낮은 앱 허용해야지 로그인 가능 <br>
https://myaccount.google.com/lesssecureapps?pli=1&rapt=AEjHL4MmWD1ouyQrMmzEva3KZMvV6ZoXD-y-oj3GkbJfwaSuRXoyFK9FY7BKbaBNxfHgGAjyRKd5k99hG3efbCEXl8q3TnXF-w

In [1]:
# 기본 준비 설정
# -*- coding:utf-8 -*-
import smtplib, email, os
# 엑셀 파일을 확인하는 명령어
import openpyxl, sys
from io import StringIO
import time 
import random

from email.header import Header
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email import encoders
from email.mime.base import MIMEBase

In [2]:
gid = input("G-Mail 로그인 계정을 입력하세요: ")
gpw = input("G-Mail 로그인 비밀번호를 입력하세요: ")

s_time = time.time()

# file_name = input('메일에 첨부할 파일명: ')
txt_files = 'data/stop_words.txt'
img_files = 'email/wordcloud.png'

# 엑셀 파일에서 목록 불어와서 자동으로 메일 보내기
wb = openpyxl.load_workbook('email/email_list.xlsx')
sheet = wb["Sheet1"]

# 엑셀파일 데이터 담기
member = { }

for i in range(2, sheet.max_row + 1):
    name = sheet.cell(row=i, column=1).value
    email = sheet.cell(row=i, column=2).value
    member[name] = email

# 서버의 승인 받기
smtp_1 = smtplib.SMTP_SSL('smtp.gmail.com',465)
smtp_1.ehlo()
# 보안수준 낮은 앱 허용해야함
smtp_1.login(gid,gpw)

count = 0 # 발송 메일 개수
complete_list = [] # 발송 성공건
fail_list = [] # 발송 실패건

for name, email in member.items():
    # 보낼 메일 내용을 직접 입력할 경우
    # msg = MIMEText('%s님 당첨되었습니다.', %name)
    
    # 만약 보낼 내용을 특정 파일에서 읽어와서 사용할 경우 
    with open('email/ment.txt', encoding='cp949') as ment:
        msgtxt=MIMEText(ment.read() %name)
        
    # txt 첨부파일 등록
    filename_txt = txt_files
    part_txt = MIMEBase('application', 'octet-stream')
    part_txt.set_payload(open(filename_txt,'rb').read())
    encoders.encode_base64(part_txt)
    part_txt.add_header('Content-Disposition','attachment:filename=%s' % os.path.basename(filename_txt))
    msg = MIMEMultipart()
    msg.attach(part_txt)
    
    # image 첨부파일 등록
    filename_img = img_files
    part_img = MIMEBase('application', ' octet-stream')
    part_img.set_payload(open(filename_img,'rb').read())
    encoders.encode_base64(part_img)
    part_img.add_header('Content-Disposition','attachment:filename=%s' % os.path.basename(filename_img))
    
    # 메일 보낼 내용 설정  
    # 텍스트 첨부파일과 이미지 첨부파일 메시지 본문 설정과 헤드 설정
    msg = MIMEMultipart()
    
    # 첨부파일
    msg.attach(part_txt)
    msg.attach(part_img)
    # 메시지
    msg.attach(msgtxt)
    # 메일 제목
    msg['Subject'] = Header(s="%s님께 알려드립니다." %name, charset='utf-8')
    
    now = time.localtime()
    s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
    
    # 메일이 잘 전송되었는지 체크
    sendmailStatus = smtp_1.sendmail('dch9610@naver.com', email, msg.as_string())
    time.sleep(random.randrange(2,6))
    
    print("="*60)
    print('%s 주소로 메일 발송 시간은 %s 입니다' %(email,s))
    print('%s 주소로 메일 발송완료'%email)
    
    complete_list.append(email)
    
    count+=1
    
    # 전송이 잘안되었을경우
    if sendmailStatus != {}:
        print('%s 주소 발송 오류:%s'%(email,sendmailStatus))
        
e_time = time.time()
t_time = e_time - s_time

# 발송 완료 명단을 파일에 저장
com_list = 'email/email_com_list.txt'

f = open(com_list, 'a', encoding='UTF-8')
f.write(str(complete_list))
f.close()

print('='*60)
print('총 소요시간 : %s초 입니다.' %t_time)
print('총 %s건 발송완료했습니다.' %count)
print('발송 완료 명단은 %s에 저장되어 있습니다.' %com_list)

smtp_1.quit()

G-Mail 로그인 계정을 입력하세요: cc
G-Mail 로그인 비밀번호를 입력하세요: c


SMTPAuthenticationError: (534, b'5.7.9 Please log in with your web browser and then try again. Learn more at\n5.7.9  https://support.google.com/mail/?p=WebLoginRequired 76sm11195935pfy.82 - gsmtp')